# 프로젝트 소개: 가족상봉 도우미 (백화점 ver.) 

- 상황 가정
    - 백화점에서 부모님 혹은 자식들을 잃어버린 상황 가정
    - 사람이 많은 환경에서 구별하기 어려운 상황 가정

- 접근
    - 자료수집
        - 인물 특징을 입력 받는다
        - 수집한 인물 특징을 가지고 이미지 자료 수집해서 라벨링
        - 라벨링된 데이터 obj (학습 데이터 폴더)와 test (validation 데이터 폴더)로 복사해서 저장
    - 학습
        - 라벨링된 데이터 로컬에서 구글 콜랩으로 받아오기
        - cfg 설정
        - obj.names, obj.data
        - train & validation 데이터 tf 형식으로 변환
        - 변환된 데이터 YOLOv4 DEEPSORT로 학습
    - detect/track
        - 어느 정도 이상의 threshold 이상으로 유사하게 detect된 인물 실시간 CCTV 영상, 위치와 함께 띄워주기:
            - detect된 인물을 detect 대신 tracking으로 넘어갈 수 있나? (tracking은 identification 또한 할 수 있어야한다)
            - detect되다말면 깜빡임 현상이 예상이 된다.

- !기타:
    - 시행착오 과정
    - 욜로와 CNN의 가장 큰 특징 차이는 detect하는 속도이다. 욜로와 CNN은 어떻게 다르길래 이런 성능의 차이가 나타나는 것일까?

# 자료수집

## 사전 준비

In [239]:
import glob
import os
import shutil 
from pathlib import Path

In [ ]:
!git clone https://github.com/theAIGuysCode/Download-Google-Images.git

In [248]:
%cd Download-Google-Images

/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images


In [250]:
!pip install -r requirements.txt
!mkdir images

  Created wheel for imutils: filename=imutils-0.5.3-py3-none-any.whl size=25850 sha256=15e56adce2b27a25de37376269a50c58f9c1cf1568bb8e9db51425a096b621d4
  Stored in directory: /Users/dy/Library/Caches/pip/wheels/fc/9c/6d/1826267c72afa51b564c9c6e0f66abc806879338bc593a2270
Successfully built imutils


### 싱글 클래스

#### 클래스 생성

In [252]:
# 클래스문 자동 작성 완성

race = input('what is the race of the person? e.g. asian ')# 주로 백인 위주로 이미지가 검색된다 
gender_and_age = input('describe the person. e.g. woman, man, boy, girl, senior? ') # e.g. man, boy, girl, grandma, senior
color_of_outfit = input('describe the most outstanding color of the persons outfit. e.g. pink ')
type_of_outfit = input('what ws the outfit? e.g. tshirt ') # e.g. top, shirt, jacket, jeans

if race not in ['white', 'caucasian']:
    class_ = '{} {} in {} {}'.format(race, gender_and_age, color_of_outfit, type_of_outfit)
else:
    class_ = '{} in {} {}'.format(gender_and_age, color_of_outfit, type_of_outfit)

what is the race of the person? e.g. asian asian
describe the person. e.g. woman, man, boy, girl, senior? woman
describe the most outstanding color of the persons outfit. e.g. pink pink
what ws the outfit? e.g. tshirt tshirt


In [253]:
%cd images

class__ = re.sub(' ', '_', class_.strip())
if class__ not in os.listdir():
    !mkdir {class__}
else:
    print('{class__} directory already exists!')

/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images


In [254]:
if 'urls.txt' in os.listdir(str(os.path.join(Path.home(), "Downloads"))):
    print('before proceeding, a urls.txt file already exists in your downloads directory')

In [255]:
# 여기서부터는 수작업
assert False

AssertionError: 

#### 생성된 클래스 이미지 자료 수집

In [258]:
print(class_)

asian woman in pink tshirt


In [256]:
# 구글 이미지에서:
# </원하는 클래스> 입력 후
# 원하는 이미지 갯수만큼 뜨게 페이지 아래로 스크롤 (디폴트값으로 구글 이미지에서 스크롤링 안했을때 뜨는 이미지 개수는 100개다)
# 개발자 모드 콘솔 탭에서 console.js 한줄 한줄씩 입력 후 urls.txt 받아오기

In [ ]:
# /**
#  * simulate a right-click event so we can grab the image URL using the
#  * context menu alleviating the need to navigate to another page
#  *
#  * attributed to @jmiserez: http://pyimg.co/9qe7y
#  *
#  * @param   {object}  element  DOM Element
#  *
#  * @return  {void}
#  */
# function simulateRightClick( element ) {
#     var event1 = new MouseEvent( 'mousedown', {
#         bubbles: true,
#         cancelable: false,
#         view: window,
#         button: 2,
#         buttons: 2,
#         clientX: element.getBoundingClientRect().x,
#         clientY: element.getBoundingClientRect().y
#     } );
#     element.dispatchEvent( event1 );
#     var event2 = new MouseEvent( 'mouseup', {
#         bubbles: true,
#         cancelable: false,
#         view: window,
#         button: 2,
#         buttons: 0,
#         clientX: element.getBoundingClientRect().x,
#         clientY: element.getBoundingClientRect().y
#     } );
#     element.dispatchEvent( event2 );
#     var event3 = new MouseEvent( 'contextmenu', {
#         bubbles: true,
#         cancelable: false,
#         view: window,
#         button: 2,
#         buttons: 0,
#         clientX: element.getBoundingClientRect().x,
#         clientY: element.getBoundingClientRect().y
#     } );
#     element.dispatchEvent( event3 );
# }

In [ ]:
# /**
#  * grabs a URL Parameter from a query string because Google Images
#  * stores the full image URL in a query parameter
#  *
#  * @param   {string}  queryString  The Query String
#  * @param   {string}  key          The key to grab a value for
#  *
#  * @return  {string}               value
#  */
# function getURLParam( queryString, key ) {
#     var vars = queryString.replace( /^\?/, '' ).split( '&' );
#     for ( let i = 0; i < vars.length; i++ ) {
#         let pair = vars[ i ].split( '=' );
#         if ( pair[0] == key ) {
#             return pair[1];
#         }
#     }
#     return false;
# }

In [ ]:
# /**
#  * Generate and automatically download a txt file from the URL contents
#  *
#  * @param   {string}  contents  The contents to download
#  *
#  * @return  {void}
#  */
# function createDownload( contents ) {
#     var hiddenElement = document.createElement( 'a' );
#     hiddenElement.href = 'data:attachment/text,' + encodeURI( contents );
#     hiddenElement.target = '_blank';
#     hiddenElement.download = 'urls.txt';
#     hiddenElement.click();
# }


In [ ]:
# /**
#  * grab all URLs va a Promise that resolves once all URLs have been
#  * acquired
#  *
#  * @return  {object}  Promise object
#  */
# function grabUrls() {
#     var urls = [];
#     return new Promise( function( resolve, reject ) {
#         var count = document.querySelectorAll(
#         	'.isv-r a:first-of-type' ).length,
#             index = 0;
#         Array.prototype.forEach.call( document.querySelectorAll(
#         	'.isv-r a:first-of-type' ), function( element ) {
#             // using the right click menu Google will generate the
#             // full-size URL; won't work in Internet Explorer
#             // (http://pyimg.co/byukr)
#             simulateRightClick( element.querySelector( ':scope img' ) );
#             // Wait for it to appear on the <a> element
#             var interval = setInterval( function() {
#                 if ( element.href.trim() !== '' ) {
#                     clearInterval( interval );
#                     // extract the full-size version of the image
#                     let googleUrl = element.href.replace( /.*(\?)/, '$1' ),
#                         fullImageUrl = decodeURIComponent(
#                         	getURLParam( googleUrl, 'imgurl' ) );
#                     if ( fullImageUrl !== 'false' ) {
#                         urls.push( fullImageUrl );
#                     }
#                     // sometimes the URL returns a "false" string and
#                     // we still want to count those so our Promise
#                     // resolves
#                     index++;
#                     if ( index == ( count - 1 ) ) {
#                         resolve( urls );
#                     }
#                 }
#             }, 10 );
#         } );
#     } );
# }


In [ ]:
# /**
#  * Call the main function to grab the URLs and initiate the download
#  */
# grabUrls().then( function( urls ) {
#     urls = urls.join( '\n' );
#     createDownload( urls );
# } );

In [260]:
!Pwd

/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images


In [279]:
# 다운로드 폴더에서 urls.txt를 images 디렉토리로 옮겨주기
path_to_download_folder = os.path.join(Path.home(), "downloads")

In [296]:
shutil.move(os.path.join(path_to_download_folder,'urls.txt'), os.path.join(os.getcwd(),'urls.txt'))

'/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/urls.txt'

In [ ]:
%cd ..

In [298]:
!python download_images.py --urls urls.txt --output images/{class__}

# 참고사항: download_images.py 중에 url로 이미지가 다운받아지지 않는 경우에는 자체적으로 이미지를 제거하는 조건문 포함
# 참고사항: 이미지 파일명을 오름차순 방식으로 숫자로 저장

[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000000.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000001.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000002.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000003.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000004.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000005.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000006.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000007.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000008.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000009.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000010.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000011.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000012.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000013.jpg
[INFO] downloaded: images/asian_woman_in_pink_tshirt/00000014.jpg
[INFO] dow

In [299]:
!rm urls.txt #다른 클래스와 겹침 방지 

In [145]:
# 여기서부터는 수작업
assert False

AssertionError: 

### 다수의 클래스

In [ ]:
!git clone https://github.com/theAIGuysCode/Download-Google-Images.git
%cd Download-Google-Images
!pip install -r requirements.txt
!mkdir images

In [87]:
# 클래스문 자동 작성 완성
classes = []

class_count = int(input('how many classes do you wish to create for your human detection? '))

for each in range(class_count):
    
    race = input('what is the race of the person? e.g. asian ')# 주로 백인 위주로 이미지가 검색된다 
    gender_and_age = input('describe the person. e.g. woman, man, boy, girl, senior? ') # e.g. man, boy, girl, grandma, senior
    color_of_outfit = input('describe the most outstanding color of the persons outfit. e.g. pink ')
    type_of_outfit = input('what ws the outfit? e.g. tshirt ') # e.g. top, shirt, jacket, jeans

    if race not in ['white', 'caucasian']:
        class_ = '{} {} in {} {}'.format(race, gender_and_age, color_of_outfit, type_of_outfit)
    else:
        class_ = '{} in {} {}'.format(gender_and_age, color_of_outfit, type_of_outfit)
    classes.append(class_)
    print(classes)

how many classes do you wish to create for your human detection? 3
what is the race of the person? e.g. asian white
describe the person. e.g. woman, man, boy, girl, senior ?woman
describe the most outstanding color of the persons outfit. e.g. pink brown
what ws the outfit? e.g. tshirt lingerie 
['woman in brown lingerie ']
what is the race of the person? e.g. asian asian
describe the person. e.g. woman, man, boy, girl, senior ?boy
describe the most outstanding color of the persons outfit. e.g. pink red
what ws the outfit? e.g. tshirt tshirt
['woman in brown lingerie ', 'asian boy in red tshirt']
what is the race of the person? e.g. asian black
describe the person. e.g. woman, man, boy, girl, senior ?person
describe the most outstanding color of the persons outfit. e.g. pink pink
what ws the outfit? e.g. tshirt grillz
['woman in brown lingerie ', 'asian boy in red tshirt', 'black person in pink grillz']


In [138]:
classes_left = classes.copy()

In [160]:
# 여기서부턴 한번만 돌리면 된다
classes = [re.sub(' ', '_', x.strip()) for x in classes]
for each in classes:
    if each not in os.listdir():
        !mkdir {each}
    else:
        print('{each} directory already exists!')

python: can't open file 'download_images.py': [Errno 2] No such file or directory
python: can't open file 'download_images.py': [Errno 2] No such file or directory
python: can't open file 'download_images.py': [Errno 2] No such file or directory


In [ ]:
if 'urls.txt' in os.listdir(str(os.path.join(Path.home(), "Downloads"))):
    print('before proceeding, a urls.txt file already exists in your downloads directory')

In [145]:
# 여기서부터는 수작업
assert False

AssertionError: 

#### 클래스 개수만큼 자료수집 반복되는 과정

- 구글 이미지를 들어간다
- 받은 클래스 입력 후 검색
- 원하는 이미지 갯수만큼 페이지에 뜨게 아래로 스크롤 (디폴트값으로 구글 이미지에서 스크롤링 안했을때 뜨는 이미지 개수는 100개다)
- 개발자 모드 콘솔 탭에서 console.js 한줄 한줄씩 입력 후 urls.txt 받아오기

In [157]:
print(classes_left[0])

black person in pink grillz


In [ ]:
# /**
#  * simulate a right-click event so we can grab the image URL using the
#  * context menu alleviating the need to navigate to another page
#  *
#  * attributed to @jmiserez: http://pyimg.co/9qe7y
#  *
#  * @param   {object}  element  DOM Element
#  *
#  * @return  {void}
#  */
# function simulateRightClick( element ) {
#     var event1 = new MouseEvent( 'mousedown', {
#         bubbles: true,
#         cancelable: false,
#         view: window,
#         button: 2,
#         buttons: 2,
#         clientX: element.getBoundingClientRect().x,
#         clientY: element.getBoundingClientRect().y
#     } );
#     element.dispatchEvent( event1 );
#     var event2 = new MouseEvent( 'mouseup', {
#         bubbles: true,
#         cancelable: false,
#         view: window,
#         button: 2,
#         buttons: 0,
#         clientX: element.getBoundingClientRect().x,
#         clientY: element.getBoundingClientRect().y
#     } );
#     element.dispatchEvent( event2 );
#     var event3 = new MouseEvent( 'contextmenu', {
#         bubbles: true,
#         cancelable: false,
#         view: window,
#         button: 2,
#         buttons: 0,
#         clientX: element.getBoundingClientRect().x,
#         clientY: element.getBoundingClientRect().y
#     } );
#     element.dispatchEvent( event3 );
# }

In [ ]:
# /**
#  * grabs a URL Parameter from a query string because Google Images
#  * stores the full image URL in a query parameter
#  *
#  * @param   {string}  queryString  The Query String
#  * @param   {string}  key          The key to grab a value for
#  *
#  * @return  {string}               value
#  */
# function getURLParam( queryString, key ) {
#     var vars = queryString.replace( /^\?/, '' ).split( '&' );
#     for ( let i = 0; i < vars.length; i++ ) {
#         let pair = vars[ i ].split( '=' );
#         if ( pair[0] == key ) {
#             return pair[1];
#         }
#     }
#     return false;
# }

In [ ]:
# /**
#  * Generate and automatically download a txt file from the URL contents
#  *
#  * @param   {string}  contents  The contents to download
#  *
#  * @return  {void}
#  */
# function createDownload( contents ) {
#     var hiddenElement = document.createElement( 'a' );
#     hiddenElement.href = 'data:attachment/text,' + encodeURI( contents );
#     hiddenElement.target = '_blank';
#     hiddenElement.download = 'urls.txt';
#     hiddenElement.click();
# }


In [ ]:
# /**
#  * grab all URLs va a Promise that resolves once all URLs have been
#  * acquired
#  *
#  * @return  {object}  Promise object
#  */
# function grabUrls() {
#     var urls = [];
#     return new Promise( function( resolve, reject ) {
#         var count = document.querySelectorAll(
#         	'.isv-r a:first-of-type' ).length,
#             index = 0;
#         Array.prototype.forEach.call( document.querySelectorAll(
#         	'.isv-r a:first-of-type' ), function( element ) {
#             // using the right click menu Google will generate the
#             // full-size URL; won't work in Internet Explorer
#             // (http://pyimg.co/byukr)
#             simulateRightClick( element.querySelector( ':scope img' ) );
#             // Wait for it to appear on the <a> element
#             var interval = setInterval( function() {
#                 if ( element.href.trim() !== '' ) {
#                     clearInterval( interval );
#                     // extract the full-size version of the image
#                     let googleUrl = element.href.replace( /.*(\?)/, '$1' ),
#                         fullImageUrl = decodeURIComponent(
#                         	getURLParam( googleUrl, 'imgurl' ) );
#                     if ( fullImageUrl !== 'false' ) {
#                         urls.push( fullImageUrl );
#                     }
#                     // sometimes the URL returns a "false" string and
#                     // we still want to count those so our Promise
#                     // resolves
#                     index++;
#                     if ( index == ( count - 1 ) ) {
#                         resolve( urls );
#                     }
#                 }
#             }, 10 );
#         } );
#     } );
# }


In [ ]:
# /**
#  * Call the main function to grab the URLs and initiate the download
#  */
# grabUrls().then( function( urls ) {
#     urls = urls.join( '\n' );
#     createDownload( urls );
# } );

In [ ]:
# 다운로드 폴더에서 urls.txt를 images 디렉토리로 옮겨주기
%cd images
path_to_download_folder = str(os.path.join(Path.home(), "Downloads"))
shutil.move(path_to_download_folder+'/urls.txt', os.getcwd()+'/{classes_left[0]}/urls.txt')

In [ ]:
!python download_images.py --urls images/{classes_left[0]}/urls.txt --output images/{classes_left[0]}
!rm images/{classes_left[0]}/urls.txt #다른 클래스와 겹침 방지

# 참고사항: download_images.py 중에 url로 이미지가 다운받아지지 않는 경우에는 자체적으로 이미지를 제거하는 조건문 포함
# 참고사항: 이미지 파일명을 오름차순 방식으로 숫자로 저장

In [140]:
# 다음 클래스 표시
classes_left.pop(0)
print(classes_left)

['black person in pink grillz']

In [142]:
# classes_left가 더이상 뜨지 않을때까지 4.1.2.1 과정 반복
print('your next class is: \n {}'.format(classes_left[0]))

your next class is: 
 black person in pink grillz


## 라벨링

In [407]:
# 수동 라벨링 GUI 모듈 다운로드: 이미 설치되어 있다면 해당 부분은 스킵해도 된다 

!git clone https://github.com/tzutalin/labelImg.git

'labelImg'에 복제합니다...
remote: Enumerating objects: 1697, done.
remote: Total 1697 (delta 0), reused 0 (delta 0), pack-reused 1697
오브젝트를 받는 중: 100% (1697/1697), 232.71 MiB | 17.77 MiB/s, 완료.
델타를 알아내는 중: 100% (1013/1013), 완료.


In [410]:
%cd labelImg

/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/labelImg


In [411]:
# 이미 설치가 되어 있다면 해당 부분은 스킵해도 된다

!pip install pyqt5
!pip install lxml
!pyrcc5 -o libs/resources.py resources.qrc

## 모듈 지원 파일
- obj.names
- obj.data
- cfg

In [327]:
!pwd

/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/labelImg


In [328]:
# classes.txt 파일 생성 후 클래스 작성 
classes_text = open(r'classes.txt', 'w')
try:
    for x in classes:
        x = re.sub('_', ' ', x)
        x += '\n'
        classes_text.write(x)
    classes_text.close()
except:
    x = class_ +'\n'
    classes_text.write(x)
    classes_text.close()

! 백업 위치를 로컬로 바꿔야할까?

In [329]:
data = open(r'data.txt', 'w')
try:
    num = len(classes)
    data.write('classes = {}\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov4/backup'.format(num))
    data.close()

except:
    data.write('classes = 1\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov4/backup')
    data.close()
os.rename('data.txt', 'obj.data')

- 라벨링 방법:
    - create\nRectBox 버튼 클릭
    - 드래그해서 관심영역을 표시 후 적합한 라벨을 클릭한다
    - 저장하기 이전에 PASCAL VOC를 눌러서 라벨링 형식을 YOLO format으로 적용한다
    - 저장 후 다음 이미지로 넘어간다

- 참고사항: 적합성이 떨어지는 이미지는 라벨링 작업을 하지 않고 스킵한다 

In [399]:
!pwd

/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images


In [403]:
%cd ..

/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images


In [401]:
%cd labelImg

/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/labelImg


In [412]:
try:
    classes_left = classes.copy()
except:
    !python labelImg.py ../images/{class__} classes.txt

[('asian woman in pink tshirt', [(106, 221), (870, 221), (870, 1428), (106, 1428)], None, None, False)]
[('asian woman in pink tshirt', [(106, 221), (870, 221), (870, 1428), (106, 1428)], None, None, False)]
Image:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt/00000000.jpg -> Annotation:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt/00000000.xml
[('asian woman in pink tshirt', [(152, 160), (877, 160), (877, 1597), (152, 1597)], None, None, False)]
Image:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt/00000001.jpg -> Annotation:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Googl

[('asian woman in pink tshirt', [(76, 70), (688, 70), (688, 1284), (76, 1284)], None, None, False)]
Image:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt/00000021.jpg -> Annotation:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt/00000021.xml
[('asian woman in pink tshirt', [(780, 306), (1155, 306), (1155, 972), (780, 972)], None, None, False)]
Image:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt/00000022.jpg -> Annotation:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt/00000022.xml
[('asian woman in pink tshirt', [(939, 36), (1391, 3

[('asian woman in pink tshirt', [(100, 39), (578, 39), (578, 417), (100, 417)], None, None, False)]
Image:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt/00000042.jpg -> Annotation:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt/00000042.xml
[('asian woman in pink tshirt', [(23, 67), (250, 67), (250, 411), (23, 411)], None, None, False)]
Image:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt/00000043.jpg -> Annotation:/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt/00000043.xml
[('asian woman in pink tshirt', [(363, 48), (730, 48), (73

In [ ]:
# try:
#     !python labelImg.py ../images/{classes_left[0]} classes.txt
#     classes_left.pop(0)
# else:
#     pass

In [377]:
%cd {'..'}

/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking
[Errno 2] No such file or directory: 'images'
/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking


In [381]:
%cd images

/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images


In [382]:
# 스킵된 이미지 삭제

try:
    for x in classes:
        %cd {x}
        txt_list = glob.glob('*.txt')
        jpg_list = [re.sub('.txt', '.jpg', each) for each in txt_list]

        whole_list = txt_list + jpg_list

        for file in os.listdir():
            if file not in whole_list:
                !rm {file}
        %cd {'..'}

except: 
    %cd {class__}
    txt_list = glob.glob('*.txt')
    jpg_list = [re.sub('.txt', '.jpg', each) for each in txt_list]

    whole_list = txt_list + jpg_list

    for file in os.listdir():
        if file not in whole_list:
            !rm {file}
    %cd {'..'}

/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/asian_woman_in_pink_tshirt
/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images


In [346]:
os.rename('classes.txt', 'obj.names')

In [353]:
# cfg 파일 생성
cfg_txt = open('yolov4-obj.txt', 'w')
list_of_lines = ['[net]\n', '# Testing\n', '#batch=1\n', '#subdivisions=1\n', '# Training\n', 'batch=64\n', 'subdivisions=16\n', 'width=416\n', 'height=416\n', 'channels=3\n', 'momentum=0.949\n', 'decay=0.0005\n', 'angle=0\n', 'saturation = 1.5\n', 'exposure = 1.5\n', 'hue=.1\n', '\n', 'learning_rate=0.001\n', 'burn_in=1000\n', 'max_batches = 6000\n', 'policy=steps\n', 'steps=4800,5400\n', 'scales=.1,.1\n', '\n', '#cutmix=1\n', 'mosaic=1\n', '\n', '#:104x104 54:52x52 85:26x26 104:13x13 for 416\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=32\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '# Downsample\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=3\n', 'stride=2\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[route]\n', 'layers = -2\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=32\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[route]\n', 'layers = -1,-7\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '# Downsample\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=3\n', 'stride=2\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[route]\n', 'layers = -2\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=64\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[route]\n', 'layers = -1,-10\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '# Downsample\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=3\n', 'stride=2\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[route]\n', 'layers = -2\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[route]\n', 'layers = -1,-28\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '# Downsample\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=3\n', 'stride=2\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[route]\n', 'layers = -2\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[route]\n', 'layers = -1,-28\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '# Downsample\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=1024\n', 'size=3\n', 'stride=2\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[route]\n', 'layers = -2\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[shortcut]\n', 'from=-3\n', 'activation=linear\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', '\n', '[route]\n', 'layers = -1,-16\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=1024\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=mish\n', 'stopbackward=800\n', '\n', '##########################\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=1024\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '### SPP ###\n', '[maxpool]\n', 'stride=1\n', 'size=5\n', '\n', '[route]\n', 'layers=-2\n', '\n', '[maxpool]\n', 'stride=1\n', 'size=9\n', '\n', '[route]\n', 'layers=-4\n', '\n', '[maxpool]\n', 'stride=1\n', 'size=13\n', '\n', '[route]\n', 'layers=-1,-3,-5,-6\n', '### End SPP ###\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=1024\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[upsample]\n', 'stride=2\n', '\n', '[route]\n', 'layers = 85\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[route]\n', 'layers = -1, -3\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=512\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=512\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[upsample]\n', 'stride=2\n', '\n', '[route]\n', 'layers = 54\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[route]\n', 'layers = -1, -3\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=256\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=256\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=128\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '##########################\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=256\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'filters=18\n', 'activation=linear\n', '\n', '\n', '[yolo]\n', 'mask = 0,1,2\n', 'anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401\n', 'classes=1\n', 'num=9\n', 'jitter=.3\n', 'ignore_thresh = .7\n', 'truth_thresh = 1\n', 'scale_x_y = 1.2\n', 'iou_thresh=0.213\n', 'cls_normalizer=1.0\n', 'iou_normalizer=0.07\n', 'iou_loss=ciou\n', 'nms_kind=greedynms\n', 'beta_nms=0.6\n', 'max_delta=5\n', '\n', '\n', '[route]\n', 'layers = -4\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=2\n', 'pad=1\n', 'filters=256\n', 'activation=leaky\n', '\n', '[route]\n', 'layers = -1, -16\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=512\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=512\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=256\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=512\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'filters=18\n', 'activation=linear\n', '\n', '\n', '[yolo]\n', 'mask = 3,4,5\n', 'anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401\n', 'classes=1\n', 'num=9\n', 'jitter=.3\n', 'ignore_thresh = .7\n', 'truth_thresh = 1\n', 'scale_x_y = 1.1\n', 'iou_thresh=0.213\n', 'cls_normalizer=1.0\n', 'iou_normalizer=0.07\n', 'iou_loss=ciou\n', 'nms_kind=greedynms\n', 'beta_nms=0.6\n', 'max_delta=5\n', '\n', '\n', '[route]\n', 'layers = -4\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=2\n', 'pad=1\n', 'filters=512\n', 'activation=leaky\n', '\n', '[route]\n', 'layers = -1, -37\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=1024\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=1024\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'filters=512\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'batch_normalize=1\n', 'size=3\n', 'stride=1\n', 'pad=1\n', 'filters=1024\n', 'activation=leaky\n', '\n', '[convolutional]\n', 'size=1\n', 'stride=1\n', 'pad=1\n', 'filters=18\n', 'activation=linear\n', '\n', '\n', '[yolo]\n', 'mask = 6,7,8\n', 'anchors = 12, 16, 19, 36, 40, 28, 36, 75, 76, 55, 72, 146, 142, 110, 192, 243, 459, 401\n', 'classes=1\n', 'num=9\n', 'jitter=.3\n', 'ignore_thresh = .7\n', 'truth_thresh = 1\n', 'random=1\n', 'scale_x_y = 1.05\n', 'iou_thresh=0.213\n', 'cls_normalizer=1.0\n', 'iou_normalizer=0.07\n', 'iou_loss=ciou\n', 'nms_kind=greedynms\n', 'beta_nms=0.6\n', 'max_delta=5\n', '\n']
cfg_txt.writelines(list_of_lines)
cfg_txt.close()

In [356]:
# cfg 파일 수정
cfg_txt = open('yolov4-obj.txt', "r")
list_of_lines = cfg_txt.readlines()

In [357]:
# #testing
# list_of_lines[2] = 'batch=1'
# list_of_lines[3] = 'subdivisions=1'
# list_of_lines[5] = '#batch=64'
# list_of_lines[6] = '#subdivisions=16'

#training
list_of_lines[2] = '#batch=1'
list_of_lines[3] = '#subdivisions=1'
list_of_lines[5] = 'batch=64'
list_of_lines[6] = 'subdivisions=16'


In [359]:
# classes, filters
try:
    classes_count = len(classes)
except:
    classes_count = 1

filters = (classes_count+5)*3

list_of_lines[962] = 'filters={}'.format(filters)
list_of_lines[969] = 'classes={}'.format(classes_count)

list_of_lines[1050] = 'filters={}'.format(filters)
list_of_lines[1057] = 'classes={}'.format(classes_count)

list_of_lines[1138] = 'filters={}'.format(filters)
list_of_lines[1145] = 'classes={}'.format(classes_count)

# width, height
width, height = 416, 416
list_of_lines[7] = 'width = {}'.format(width)
list_of_lines[8] = 'height = {}'.format(height)

# max_batches, steps
if classes_count >=3:
    max_batches = classes_count*2000 # no. of classes * 2000, min 6000
else:
    max_batches = 6000

steps1, steps2 = np.round(0.8*max_batches), np.round(0.9*max_batches)

list_of_lines[19] = 'max_batches = {}'.format(max_batches)
list_of_lines[20] = 'steps = {}, {}'.format(steps1, steps2)

In [360]:
a_file = open('yolov4-obj.txt', "w")
a_file.writelines(list_of_lines)
a_file.close()

In [389]:
os.rename('yolov4-obj.txt', 'yolov4-obj.cfg')

## obj & test 디렉토리로 자료 정리

In [361]:
try:
    !mkdir obj
    !mkdir test
except:
    print('directories already exist')
    pass

In [ ]:
# 여기서 train만 시킬건지 validation dataset도 만들건지 선택
assert False

### train 데이터만

In [195]:
try:
    for each in classes:
        path = os.path.join(os.getcwd(),each)
        jpg_files = [file for file in os.listdir(path) if file.endswith('.jpg')]
        txt_files = [re.sub('.jpg', '.txt', file) for file in jpg_files]

        train_count  = len(jpg_files)
        rndnums = list(random.sample(range(0, len(jpg_files)), len(jpg_files)))

        train_file_index = rndnums[0:int(train_count)+1]
        train_file_name = [jpg_files[i] for i in train_file_index]
        train_file_name.extend([txt_files[i] for i in train_file_index])

        # train_files
        for x in train_file_name:
            shutil.copyfile(os.path.join(path, x), os.path.join(os.getcwd(), 'obj/{x}'))

except:
    path = os.path.join(os.getcwd(),class__)
    jpg_files = [file for file in os.listdir(path) if file.endswith('.jpg')]
    txt_files = [re.sub('.jpg', '.txt', file) for file in jpg_files]

    train_count  = len(jpg_files)
    rndnums = list(random.sample(range(0, len(files)), len(files)))

    train_file_index = rndnums[0:int(train_count)+1]
    train_file_name = [jpg_files[i] for i in train_file_index]
    train_file_name.extend([txt_files[i] for i in train_file_index])

    # train_files
    for x in train_file_name:
        shutil.copyfile(os.path.join(path, x), os.path.join(os.getcwd(), 'obj/{x}'))

enter valid size (between 0~1)0.8


FileNotFoundError: [Errno 2] No such file or directory: '/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/{each}'

In [215]:
assert False

AssertionError: 

### train & validation 데이터

In [362]:
os.getcwd()

'/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images'

In [363]:
!ls

asian_woman_in_pink_tshirt obj.names
obj                        test
obj.data                   yolov4-obj.txt


In [387]:
# test_size = float(input('enter valid size (between 0~1)\n'))
test_size = 0.2
train_size = 1-test_size

obj = os.path.join(os.getcwd(), 'obj')
test = os.path.join(os.getcwd(), 'test')

try:
    for each in classes:
        path = os.path.join(os.getcwd(),each)
        jpg_files = [file for file in os.listdir(path) if file.endswith('.jpg')]
        txt_files = [re.sub('.jpg', '.txt', file) for file in jpg_files]

        train_count  = np.round(train_size*len(jpg_files))
        rndnums = list(random.sample(range(0, len(jpg_files)), len(jpg_files)))

        train_file_index = rndnums[0:int(train_count)+1]
        train_file_name = [jpg_files[i] for i in train_file_index]
        train_file_name.extend([txt_files[i] for i in train_file_index])

        valid_file_index = rndnums[int(train_count)+1:]
        valid_file_name = [jpg_files[i] for i in valid_file_index]
        valid_file_name.extend([txt_files[i] for i in valid_file_index])

        # train_files
        for x in train_file_name:
            shutil.copyfile(os.path.join(path, x), os.path.join(os.getcwd(), 'obj/{x}'))

        # valid_files
        for z in valid_file_name:
            shutil.copyfile(os.path.join(path, z), os.path.join(os.getcwd(), 'test/{z}'))

except:
    path = os.path.join(os.getcwd(),class__)
    jpg_files = [file for file in os.listdir(path) if file.endswith('.jpg')]
    txt_files = [re.sub('.jpg', '.txt', file) for file in jpg_files]

    train_count  = np.round(train_size*len(jpg_files))
    rndnums = list(random.sample(range(0, len(jpg_files)), len(jpg_files)))

    train_file_index = rndnums[0:int(train_count)+1]
    train_file_name = [jpg_files[i] for i in train_file_index]
    train_file_name.extend([txt_files[i] for i in train_file_index])

    valid_file_index = rndnums[int(train_count)+1:]
    valid_file_name = [jpg_files[i] for i in valid_file_index]
    valid_file_name.extend([txt_files[i] for i in valid_file_index])

    # train_files
    for x in train_file_name:
        shutil.copyfile(os.path.join(path, x), os.path.join(obj, '{}'.format(x)))

    # valid_files
    for z in valid_file_name:
        shutil.copyfile(os.path.join(path, z), os.path.join(test, '{}'.format(z)))

In [393]:
!mkdir data

In [394]:
!ls

asian_woman_in_pink_tshirt obj.data
data                       obj.names
images.zip                 test
obj                        yolov4-obj.cfg


In [396]:
for each in ['obj.data', 'obj.names', 'yolov4-obj.cfg', 'obj', 'test']:
    shutil.move(os.path.join(os.getcwd(),each), os.path.join(os.getcwd(), 'data', each))

In [397]:
shutil.make_archive("objs","zip", os.path.join(os.getcwd(), 'data'))

'/Users/dy/Documents/dss14/dss_workspace/project/deep_learning_projects/deeplearning-repo-1/human_tracking/Download-Google-Images/images/objs.zip'

# YOLOv4 Object Detector w/ Darknet

구글콜랩 주소 

# 마치며

## !YOLO vs CNN

성능차이 원인에 대한 궁금증이 생겼다

## !시행착오 과정

- 깃헙에서 theAIGuysCode 참조:
    - https://github.com/theAIGuysCode/Download-Google-Images
    > 클래스별로 이미 라벨링된 이미지 데이터셋을 가져와서 학습시켜봤으나 클래스가 구체적인 특징을 갖지 않아서 recall 수치가 떨어짐
    
    - 보완: 구체적인 특징을 가진 이미지 직접 라벨링 후 학습 데이터로 사용
    > 몇가지 질문으로 인물의 특징을 가지고 구글 이미지에서 학습할 수 있는 이미지 자료를 가져와서https://github.com/tzutalin/labelImg 에서 제공하는 라벨링 툴을 이용함

In [ ]:
인상착의 입력해서 OIDv6 데이터셋을 사용해본 결과 recall이 매우 저조 
-> 직접 입력해서 직접 라벨링 하는 방식으로 수정

google images에서 자동으로 검색어 입력 후 검색
검색된 이미지 최대 x개까지 퍼오는 기능 구현
라벨링 툴 불러와서 수동 라벨링 진행
(라벨링 yolo 학습모델에 맞게 좌표 변환 후 txt 파일로 자동 저장)

기존 데이터가 들어 있는 디렉토리들 (obj & test)와 통합
generate_train
generate_test

욜로 모델에 학습 

In [ ]:
# - https://github.com/theAIGuysCode/Download-Google-Images.git에서 제공하는 download_images.py 모듈로 구글 이미지 다운로드
# - 클래스가 여러개인 경우 images 디렉토리에서 클래스별로 디렉토리 만들어주고 라벨링 툴 매번 새로 열어줘야함

# - 1차 보완: 한 폴더에서 여러개의 클래스를 누적변경되게 download_images.py의 line 17부터 total 객체 생성 코드 수정

# try:
#     total = len(os.listdir(args['output']))
# except ExplicitException:
#     try:
#         pwd_ = os.getcwd()
#         total = len(os.listdir(pwd_ + re.sub('./', '', args.output))
#     except ExplicitException:
#         try:
#             pwd_ = os.getcwd()
#             pwd_ = '/'.join(pwd_.split('/')[:-1])
#             total = len(os.listdir(pwd_ + re.sub('../', '', args.output))
#         except:
#             print('enter the whole correct output path')

# - 그러나 자료의 분류가 불가능해서 동일한 특징을 가진 다른 인물을 찾기 위해서 데이터를 학습시킬때 구글 이미지에서 또 크롤링해야하는 번거로움 발생

# - 2차 보완: download_images.py 원상태 복구 후 클래스별로 디렉토리 생성해주고 

## 독자들에게 질문을 남겨도 되겠다

In [160]:
# 여기서부턴 한번만 돌리면 된다
classes = [re.sub(' ', '_', x.strip()) for x in classes]
for each in classes:
    if each not in os.listdir():
        !mkdir {each}
        !python download_images.py --urls images/{each}/urls.txt --output images/{each}
    else:
        print('{each} directory already exists!')

python: can't open file 'download_images.py': [Errno 2] No such file or directory
python: can't open file 'download_images.py': [Errno 2] No such file or directory
python: can't open file 'download_images.py': [Errno 2] No such file or directory


In [ ]:
Steps to perform:

Query your intended Google search
Scroll down through images until they become unrelated to your query or until you've passed enough images for your dataset
Right click and hit "Inspect" and then navigate to "Console" tab
One by one enter the lines from console.js into the console window and run them
Move urls.txt from Downloads folder to Download-Google-Images folder
Create an "images" folder where your images will be downloaded
Run follwing two python commands as follows:
pip install -r requirements.txt
python download_images.py --urls urls.txt --output images
You should now have all your images inside your images folder!

## 사이트에 뜬 구글 이미지 url 셀레니움으로 자동 크롤링 모듈 생성: 미완성 

In [ ]:
# 셀레니움으로 구글 이미지 서치란에 찾고 검색어 입력 후 클릭 
# 이미지를 퍼오고 싶은 만큼 (횟수 혹은 시간초) 페이지를 아래로 스크롤하는 명령 입력 (스크롤링 타이머 혹은 양 측정 결과 얼만큼의 이미지가 퍼와지더라도 쓸것)
# 오른쪽 클릭 눌러서 inspect 클릭

In [44]:
# 콘솔탭 클릭해서 하단 입력란에 마지막 쿼리문까지 한줄씩 입력 후 엔터

from selenium import webdriver

url = 'https://www.youtube.com/watch?v=EGQyDla8JNU&t=599s'

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized");
options.add_argument("disable-infobars");
options.add_argument("--disable-notifications");
options.add_argument("--auto-open-devtools-for-tabs");

driver = webdriver.Chrome(executable_path='./chromedriver', chrome_options=options)
driver.get(url=url)

/Users/dy/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [62]:
!pip install webdriver_manager

In [67]:
from webdriver_manager.chrome import ChromeDriverManager

In [76]:
# 크롬드라이버 자동 업데이트 모듈
options = webdriver.ChromeOptions()
options.add_argument('--remote-debugging-port=8889')
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/dy/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


WebDriverException: Message: chrome not reachable


In [ ]:
import pychrome
dev_tools = pychrome.Browser(url='http://localhost:8889')
dev_tools.list_tab()

In [ ]:
import pychrome
dev_tools = pychrome.Browser(url='http://localhost:8889')
tab = dev_tools.list_tab()[0]
tab.start()

driver.get(url)

# tab.call_method('Network.emulateNetworkConditions',
#                offline=False,
#                latency=100,
#                downloadThroughput=93750,
#                uploadThroughput=31250,
#                connectionType='cellular3g')

# tab.call_method('Console.enable', _timeout=20)
# # output_on_start with no brackets : not running the function but passing the function object
# tab.set_listener('Network.requestWillBeSent', output_on_start)
# tab.set_listener('Network.responseRecieved', output_on_end)
tab.call_method('Console.enable', _timeout=20)
tab.set_listener('Console.ConsoleMessage')

driver.get(url)

In [ ]:
https://chromedevtools.github.io/devtools-protocol/tot/Console/#type-ConsoleMessage

In [65]:
import warnings; warnings.filterwarnings('ignore')

In [45]:
element = driver.find_elements_by_css_selector('#country-code')

In [48]:
element

[<selenium.webdriver.remote.webelement.WebElement (session="2f31e460826bd7c19ffdcaa9e56055ee", element="3af25591-d405-4de9-8cce-8a7e24d8c220")>]

In [46]:
location = element[0].location
size = element[0].size
print(location, size)

{'x': 140, 'y': 8} {'height': 40, 'width': 0}


In [47]:
location2 = element[1].location
size2 = element[1].size
print(location, size)

IndexError: list index out of range

In [49]:
png = driver.get_screenshot_as_png()
driver.quit()

In [50]:
from PIL import Image
from io import BytesIO

In [51]:
im = Image.open(BytesIO(png))

left = location['x']
top = location['y']
right = location['x'] + size['width']
bottom = location['y'] + size['height']

im = im.crop((left, top, right, bottom))


In [57]:
im = Image.open(BytesIO(png))

In [58]:
import matplotlib.pyplot as plt

im.show()

In [ ]:
from PIL import Image
from io import BytesIO

fox = webdriver.Firefox()
fox.get('http://stackoverflow.com/')

# now that we have the preliminary stuff out of the way time to get that image :D
element = fox.find_element_by_id('hlogo') # find part of the page you want image of
location = element.location
size = element.size
png = fox.get_screenshot_as_png() # saves screenshot of entire page
fox.quit()

im = Image.open(BytesIO(png)) # uses PIL library to open image in memory

left = location['x']
top = location['y']
right = location['x'] + size['width']
bottom = location['y'] + size['height']


im = im.crop((left, top, right, bottom)) # defines crop points
im.save('screenshot.png') # saves new cropped image

In [30]:
element

[<selenium.webdriver.remote.webelement.WebElement (session="3aece7358f3515058daa696b53b33af1", element="c50fff67-b5ab-45c5-a4a8-79ac38c78cfc")>,
 <selenium.webdriver.remote.webelement.WebElement (session="3aece7358f3515058daa696b53b33af1", element="926d95a5-d680-448e-9127-6f215cd72019")>]

In [33]:
driver.quit()

In [ ]:
# when urls.txt is downloaded move txt file to Download-Google-Images repository
# add new folder called images unless it already exists
# for multiple classes you have to make sub folders for each different classes in the images folder

In [ ]:
!python download_images.py --urls urls.txt --output #images/hotdog example in the tutorial video

In [ ]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

ChromeOptions options = new ChromeOptions();
options.AddArgument("--start-maximized");
options.AddArguments("disable-infobars");
options.AddArguments("--disable-notifications");
options.AddArguments("--auto-open-devtools-for-tabs");
driver = new ChromeDriver(DrivePath, options, TimeSpan.FromSeconds(100));

#browser exposes an executable file
#Through Selenium test we will invoke the executable file which will then
#invoke actual browser


driver = webdriver.Chrome(executable_path="C:\\chromedriver.exe")
# to maximize the browser window
driver.maximize_window()
#get method to launch the URL
driver.get("https://www.tutorialspoint.com/about/about_careers.htm")
#to refresh the browser
driver.refresh()
# identifying the source element
source= driver.find_element_by_xpath("//*[text()='Company']");
# action chain object creation
action = ActionChains(driver)
# right click operation and then perform
action.context_click(source).perform()
#to close the browser
driver.close()

In [ ]:
https://stackoverflow.com/questions/37683576/how-do-you-automatically-open-the-chrome-devtools-tab-within-selenium-c